<a href="https://colab.research.google.com/github/m-wessler/nbm-verification/blob/main/nbm_textfile_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
import re
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import glob

# Supported file types
supported_file_types = ['nbp', 'nbe', 'nbs', 'nbx']

# Ask the user to choose the file type
file_type = input(f"Choose the file type to process ({', '.join(supported_file_types)}): ").strip().lower()

if file_type not in supported_file_types:
    raise ValueError(f"Unsupported file type: {file_type}. Supported types are: {', '.join(supported_file_types)}")

# Glob for all matching *.txt files
file_pattern = f"*{file_type}*.txt"
matching_files = glob.glob(file_pattern)

if not matching_files:
    print(f"No files found matching the pattern: {file_pattern}")
    exit()

print(f"Found {len(matching_files)} file(s) matching the pattern: {file_pattern}")
for idx, file in enumerate(matching_files, 1):
    print(f"{idx}. {file}")

# Process each matching file
for input_file_path in matching_files:
    print(f"\nProcessing file: {input_file_path}")

    # Extract the base name of the input file
    input_file_name = os.path.basename(input_file_path).split('.')[0]  # Remove extension for output filename

    # Read the file
    with open(input_file_path, "r") as file:
        lines = file.read().splitlines()

    # Split the content into blocks based on blank rows
    blocks = []
    current_block = []
    for line in lines:
        if not line.strip():  # Blank line indicates end of a block
            if current_block:
                blocks.append(current_block)
                current_block = []
        else:
            current_block.append(line)

    # Add the last block if it wasn't added
    if current_block:
        blocks.append(current_block)

    # Process metadata from the first row of each block, discarding invalid blocks
    valid_blocks = []
    output_init_time = None  # Will store the init_time for the output filename
    for block in blocks:
        if block:  # Ensure the block isn't empty
            metadata_line = block[0]  # First row contains metadata
            parts = re.split(r"\s{2,}", metadata_line.strip())  # Split by 2+ spaces

            # Validate metadata: ensure we have a proper init time (last two parts of the metadata line)
            if len(parts) >= 2:
                site_id = parts[0].split()[0]  # Extract SITE_ID (first part)
                init_time = " ".join(parts[-2:])  # Combine date and time (last two parts)

                # Check if the init_time is in the correct format
                if re.match(r"^\d{1,2}/\d{1,2}/\d{4} \d{4} UTC$", init_time):
                    valid_blocks.append((site_id, init_time, block))  # Store valid blocks
                    if output_init_time is None:
                        # Format init_time for the output filename (yyyymmddhh)
                        init_time_dt = datetime.strptime(init_time, "%m/%d/%Y %H%M %Z")
                        output_init_time = init_time_dt.strftime("%Y%m%d%H")
                else:
                    print(f"Discarding block due to invalid init time: {metadata_line}")
            else:
                print(f"Discarding block due to insufficient metadata: {metadata_line}")

    # Prepare the DataFrame
    df = pd.DataFrame()

    # Start parsing data rows for each valid block
    for site_id, init_time, block in valid_blocks:
        print(f"Processing block for Site ID: {site_id}, Init Time: {init_time}")

        # Convert init_time to a datetime object
        init_time_dt = datetime.strptime(init_time, "%m/%d/%Y %H%M %Z")

        # Skip the first three rows (metadata + headers)
        data_rows = block[3:]

        # Initialize a dictionary to store the data for the current block
        block_data = {"site_id": site_id, "init_time": init_time_dt}

        if file_type == 'nbp':
            # Parsing logic for 'nbp' type
            for row in data_rows:
                # Extract the variable name (first 5 characters after 1 space)
                variable_name = row[1:6].strip()

                # Check if the row contains bad data (-459)
                if "-459" in row:
                    # Skip the row and backfill with NaN
                    parsed_data = [np.nan] * 17  # Backfill the row with 17 NaN values
                else:
                    # Parse the data columns
                    parsed_data = [
                        row[7:10].strip(), row[11:14].strip(), row[15:18].strip(), row[19:22].strip(),
                        row[23:26].strip(), row[27:30].strip(), row[31:34].strip(), row[35:38].strip(),
                        row[39:42].strip(), row[43:46].strip(), row[47:50].strip(), row[51:54].strip(),
                        row[55:58].strip(), row[59:62].strip(), row[63:66].strip(), row[67:70].strip(),
                        row[71:74].strip()
                    ]

                    # Replace blank values with np.nan
                    parsed_data = [np.nan if val == "" else val for val in parsed_data]

                # Add data to block_data
                block_data[variable_name] = parsed_data

        elif file_type == 'nbs':
            # Parsing logic for 'nbs' type (handling FHR with 23 items)
            fhr_processed = False  # Track when the FHR row is processed

            for row in data_rows:
                # Extract the variable name (first 3 characters)
                variable_name = row[1:4].strip()

                # Check if the row contains bad data (-459)
                if "-459" in row:
                    # Skip the row and backfill with NaN
                    parsed_data = [np.nan] * 23  # Backfill the row with 23 NaN values
                else:
                    # Special handling for the FHR row
                    if not fhr_processed and variable_name == "FHR":
                        # Parse only 23 items for the FHR row
                        parsed_data = [
                            row[i:i+3].strip()
                            for i in range(5, 5 + 23 * 3, 3)  # Parse exactly 23 items
                        ]
                        fhr_processed = True  # Mark the FHR row as processed
                    else:
                        # Parse the data columns dynamically (e.g., [5:8], [8:11], [11:14], ...)
                        parsed_data = [
                            row[i:i+3].strip()
                            for i in range(5, len(row), 3)  # Start from index 5 and increment by 3
                        ]

                    # Replace blank values with np.nan
                    parsed_data = [np.nan if val == "" else val for val in parsed_data]

                # Add data to block_data
                block_data[variable_name] = parsed_data

        # Convert block_data to a DataFrame
        block_df = pd.DataFrame(block_data)

        # Calculate valid_time based on FHR
        if "FHR" in block_df.columns:
            block_df["FHR"] = pd.to_numeric(block_df["FHR"], errors="coerce")  # Ensure FHR is numeric
            block_df["valid_time"] = block_df["FHR"].apply(
                lambda x: init_time_dt + timedelta(hours=x) if not pd.isna(x) else np.nan
            )

        # Ensure numeric columns are properly set to int or float, filling NaN where necessary
        for column in block_df.columns:
            if column not in ["site_id", "init_time", "valid_time"]:  # Skip non-numeric columns
                block_df[column] = pd.to_numeric(block_df[column], errors="coerce")

        # Concatenate block DataFrame with the main DataFrame
        df = pd.concat([df, block_df], ignore_index=True)

    # Set MultiIndex [init_time, valid_time, site_id]
    df.set_index(["init_time", "valid_time", "site_id"], inplace=True)

    # Generate the output filename
    if output_init_time:
        output_file_name = f"{input_file_name}_{output_init_time}.csv"
    else:
        output_file_name = f"{input_file_name}_output.csv"  # Fallback if no valid init_time is found
    output_file_path = os.path.join("/content", output_file_name)

    # Save the DataFrame to a CSV file
    df.to_csv(output_file_path)
    print(f"Data saved to {output_file_path}")

Choose the file type to process (nbp, nbe, nbs, nbx): nbp
Found 1 file(s) matching the pattern: *nbp*.txt
1. blend_nbptx.t13z.txt

Processing file: blend_nbptx.t13z.txt
Discarding block due to insufficient metadata: 1
Processing block for Site ID: 086092, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 188557, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 220792, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 2A1, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 2M2, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 356033, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 359588, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 382730, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 41001, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 41002, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: TROM6, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: TROM8, Init T

In [147]:
df.loc[(slice(None), slice(None), "YLSU1")]['Q24P7']

init_time            valid_time         
2024-05-14 13:00:00  2024-05-15 12:00:00    NaN
                     2024-05-16 00:00:00    NaN
                     2024-05-16 12:00:00    0.0
                     2024-05-17 00:00:00    NaN
                     2024-05-17 12:00:00    0.0
                     2024-05-18 00:00:00    NaN
                     2024-05-18 12:00:00    0.0
                     2024-05-19 00:00:00    NaN
                     2024-05-19 12:00:00    0.0
                     2024-05-20 00:00:00    NaN
                     2024-05-20 12:00:00    0.0
                     2024-05-21 00:00:00    NaN
                     2024-05-21 12:00:00    2.0
                     2024-05-22 00:00:00    NaN
                     2024-05-22 12:00:00    1.0
                     2024-05-23 00:00:00    NaN
                     2024-05-23 12:00:00    0.0
Name: Q24P7, dtype: float64